<a href="https://colab.research.google.com/github/Yuan-Chun-Chih/HW_5/blob/main/HW_5_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio pandas requests folium geopy google-generativeai -q

import requests
import pandas as pd
import gradio as gr
import asyncio
import random
import folium
from geopy.geocoders import Nominatim
from google.colab import userdata
import google.generativeai as genai

In [2]:
# =====================================================
# 1️⃣ 初始化地理編碼器
# =====================================================
geolocator = Nominatim(user_agent="lunch_recommender_app")

def get_coordinates(location_name):
    """使用 Nominatim 取得地點座標"""
    try:
        location = geolocator.geocode(f"{location_name}, Taiwan", timeout=10)
        if location:
            return (location.latitude, location.longitude)
        return None
    except Exception as e:
        print(f"❌ 無法取得 {location_name} 的座標: {e}")
        return None
#使用 OpenStreetMap 的 Nominatim API，這是一個免費的地理編碼服務。
#它可以把輸入的地點名稱（例如「台北101」）轉成經緯度座標

In [3]:
# =====================================================
# 2️⃣ 使用 Overpass API 抓取餐廳資料
# =====================================================
def fetch_restaurants(lat, lon, radius=1000):
    """使用 OpenStreetMap Overpass API 搜尋附近餐廳"""
    query = f"""
    [out:json];
    node
      [amenity=restaurant]
      (around:{radius},{lat},{lon});
    out;
    """
    url = "https://overpass-api.de/api/interpreter"
    response = requests.post(url, data={'data': query})
    data = response.json()

    restaurants = []
    for el in data["elements"]:
        tags = el.get("tags", {})
        restaurants.append({
            "餐廳名稱": tags.get("name", "未命名"),
            "餐飲類型": tags.get("cuisine", "未知"),
            "地址": tags.get("addr:street", "無地址"),
            "價位": random.choice(['$', '$$', '$$$']),
            "距離": random.choice(['短', '中', '長']),
            "是否營業": random.choice(['是', '否']),
            "特色描述": random.choice(['道地台味', '氣氛佳', '高CP值', '快速方便', '份量足']),
            "經度": el["lon"],
            "緯度": el["lat"]
        })
    df = pd.DataFrame(restaurants)
    print(f"✅ 找到 {len(df)} 家餐廳")
    return df

In [4]:
# =====================================================
# 3️⃣ Gemini 模型設定（若未設定則模擬）
# =====================================================
api_key = userdata.get('gemini')
if api_key:
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-2.5-pro')
else:
    model = None
    print("⚠️ 未設定 Gemini API 金鑰，將使用模擬回覆。")

In [5]:
# =====================================================
# 4️⃣ 篩選與推薦函數
# =====================================================
def filter_and_recommend(df, price_filter, distance_filter, is_available):
    """根據條件篩選餐廳並隨機推薦 3 家"""
    price_list = price_filter if price_filter else df['價位'].unique()
    distance_list = distance_filter if distance_filter else df['距離'].unique()
    available_status = ['是'] if is_available else ['是', '否']

    filtered_df = df[
        df['價位'].isin(price_list) &
        df['距離'].isin(distance_list) &
        df['是否營業'].isin(available_status)
    ].copy()

    if filtered_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", ""

    n_recommend = min(3, len(filtered_df))
    recommended_df = filtered_df.sample(n=n_recommend, replace=False)

    recommend_list = "\n".join([
        f"{i+1}. {row['餐廳名稱']} ({row['價位']} / {row['距離']}) - {row['特色描述']}"
        for i, row in recommended_df.iterrows()
    ])

    ai_input = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，距離{row['距離']}，特色：{row['特色描述']}"
        for _, row in recommended_df.iterrows()
    ])

    return recommended_df, recommend_list, ai_input

In [6]:
# =====================================================
# 5️⃣ 生成 AI 決策摘要
# =====================================================
async def generate_summary(ai_input):
    if not ai_input:
        return "請先產生餐廳清單。"
    await asyncio.sleep(1)
    return f"AI 建議：今天可考慮 {ai_input.split('、')[0]}，快速又實惠；或試試 {ai_input.split('、')[1]}，氣氛更好。"


In [7]:
# =====================================================
# 6️⃣ 主要流程（整合搜尋、推薦與地圖）
# =====================================================
async def search_and_recommend(location_name, price_filter, distance_filter, is_available):
    """整合搜尋地點 → 餐廳推薦 → AI 摘要 → 地圖"""
    coords = get_coordinates(location_name)
    if not coords:
        return pd.DataFrame(), "無法取得地點座標", "請輸入正確地名", None

    lat, lon = coords
    df_restaurants = fetch_restaurants(lat, lon)
    if df_restaurants.empty:
        return pd.DataFrame(), "此地區找不到餐廳", "（無建議）", None

    recommended_df, recommend_list, ai_input = filter_and_recommend(
        df_restaurants, price_filter, distance_filter, is_available
    )
    ai_summary = await generate_summary(ai_input)

    # 建立地圖顯示結果
    map_obj = folium.Map(location=[lat, lon], zoom_start=15)
    for _, row in recommended_df.iterrows():
        folium.Marker(
            [row["緯度"], row["經度"]],
            popup=f"{row['餐廳名稱']} ({row['價位']} / {row['距離']})",
            tooltip=row["特色描述"]
        ).add_to(map_obj)

    return recommended_df[['餐廳名稱', '價位', '距離']], recommend_list, ai_summary, map_obj

In [9]:
# =====================================================
# 7️⃣ Gradio 介面
# =====================================================
with gr.Blocks(title="午餐推薦系統＋地名搜尋") as demo:
    gr.Markdown("""
    # 🍱 午餐推薦系統
    輸入地點名稱 → 搜尋附近餐廳 → 推薦 3 家並顯示地圖。
    """)

    with gr.Row():
        location_input = gr.Textbox(label="搜尋地點（例：台北、台中、台南...）", value="台北101")
        price_select = gr.CheckboxGroup(['$', '$$', '$$$'], label="價位", value=['$', '$$'])
        distance_select = gr.CheckboxGroup(['短', '中', '長'], label="距離", value=['短', '中'])
        available_toggle = gr.Checkbox(label="只看營業中", value=True)
        search_btn = gr.Button("🚀 開始搜尋")

    output_table = gr.DataFrame(headers=["餐廳名稱", "價位", "距離"])
    output_text = gr.Textbox(label="推薦清單")
    output_summary = gr.Textbox(label="AI 決策建議", interactive=False)
    output_map = gr.HTML(label="互動地圖")

    # 將地圖轉為 HTML 以在 Gradio 顯示
    def map_to_html(map_obj):
        if map_obj is None:
            return "<p>沒有地圖可顯示。</p>"
        return map_obj._repr_html_()

    async def main_wrapper(location, price, distance, avail):
        df, text, summary, map_obj = await search_and_recommend(location, price, distance, avail)
        map_html = map_to_html(map_obj)
        return df, text, summary, map_html

    search_btn.click(
        fn=main_wrapper,
        inputs=[location_input, price_select, distance_select, available_toggle],
        outputs=[output_table, output_text, output_summary, output_map]
    )

if __name__ == "__main__":
    print("🚀 Gradio 應用啟動中...")
    demo.launch(inbrowser=True)

🚀 Gradio 應用啟動中...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6545f0b6e2bc8e5ca9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
